In [1]:
import pandas as pd
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import random
import torch
from tqdm import tqdm
import evaluate

sns.set_style('ticks')

# Path dataset (sesuaikan dengan lokasi dataset Anda di Colab)
DATASET_ROOT = './indosum'

# Buat folder jika dataset belum ada
if not os.path.exists(DATASET_ROOT):
    os.makedirs(DATASET_ROOT)

# Pastikan file dataset diunggah ke folder ini sebelum menjalankan kode
files_id_dir = os.listdir(DATASET_ROOT)
train_files = []

for filename in files_id_dir:
    if 'train' in filename:
        train_files.append(filename)
        
# Fungsi untuk memuat data JSON Lines
def load_file_to_json_list(filename):
    file = os.path.join(DATASET_ROOT, filename)

    data = []
    with open(file, 'r') as f:
        # Read the entire file content
        file_content = f.read()
        
        # Split the content into individual JSON objects
        json_list = file_content.splitlines() 
        
        for json_str in tqdm(json_list, desc=f'Loading data {filename}'):
            # Skip empty lines
            if json_str.strip(): 
                try:
                    d = json.loads(json_str)
                    data.append(d)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    print(f"Problematic JSON string: {json_str}")
                    # You might want to handle the error, e.g., skip the line or try to fix the JSON
                    
    return data

# Fungsi untuk memproses label menjadi string JSON
def label_to_dict_str(label_list):
    label_dict = {}  # key = paragraph_id : value = label list 
    for i, label in enumerate(label_list[:]):
        label_dict[i] = label

    json_str = json.dumps(label_dict)
    num = len(label_dict)
    return json_str, num

# Fungsi untuk memproses paragraph menjadi string JSON
def paragraph_to_dict_str(paragraph_list):
    paragraph_dict = {}  # key = paragraph_id : value = paragraph list 
    for i, paragraph in enumerate(paragraph_list):
        new_paragraph = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            new_paragraph.append(sentence)
        paragraph_dict[i] = new_paragraph

    json_str = json.dumps(paragraph_dict)
    num = len(paragraph_dict)
    return json_str, num
    
# Fungsi untuk mengubah paragraf menjadi string teks
def paragraph_to_text(raw_paragraph_list):
    new_paragraph_list = []
    for i, paragraph in enumerate(raw_paragraph_list):
        paragraph_list = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            paragraph_list.append(sentence)

        new_paragraph = ' '.join(paragraph_list)
        new_paragraph_list.append(new_paragraph)

    paragraph_str = ' '.join(new_paragraph_list)
    return paragraph_str

# Fungsi untuk memproses summary menjadi string JSON
def summary_to_dict_str(summary_list):
    summary_dict = {}  # key = summary_id : value = summary sentence 
    for i, summary in enumerate(summary_list):
        summary_dict[i] = ' '.join(summary)

    json_str = json.dumps(summary_dict)
    num = len(summary_dict)
    return json_str, num
# Fungsi untuk mengubah summary menjadi string teks
def summary_to_text(raw_summary_list):
    summary_list = []
    for i, summary in enumerate(raw_summary_list):
        summary_list.append(' '.join(summary))

    summary_str = ' '.join(summary_list)
    return summary_str

# Fungsi untuk mengubah data JSON
def alter_json_data(json_list_data, filename=''):
    new_json_list = []
    for json_data in tqdm(json_list_data, desc=f'Altering json data {filename}'):
        json_data = json_data.copy()
        json_data['gold_labels'], _ = label_to_dict_str(json_data['gold_labels'])
        json_data['news_text'] = paragraph_to_text(json_data['paragraphs'])
        json_data['paragraphs'], num_paragraph = paragraph_to_dict_str(json_data['paragraphs'])
        json_data['num_of_paragraphs'] = num_paragraph
        json_data['summary_text'] = summary_to_text(json_data['summary'])
        json_data['summary'], num_summary = summary_to_dict_str(json_data['summary'])
        json_data['num_of_summary'] = num_summary

        new_json_list.append(json_data)
    
    return new_json_list

# Fungsi untuk membuat dataset dari JSON Lines
def create_dataset(jsonl):
    header = list(jsonl[0].keys())
    dataset_list = []
    for json_data in jsonl:
        row = []
        for h in header:
            row.append(json_data[h])
        dataset_list.append(row)
    
    return header, dataset_list

# Fungsi untuk membuat dataset dari file JSON Lines
def create_dataset_from_files(file_list):
    df_header = None
    dataset_list = []
    for filename in file_list:
        json_l = load_file_to_json_list(filename)
        new_json_l = alter_json_data(json_l, filename)
        header, dataset_part = create_dataset(new_json_l)
        
        if not df_header: df_header = header
        dataset_list.extend(dataset_part)
        
    df_full = pd.DataFrame().from_records(dataset_list)
    df_full = df_full.rename(columns=dict(enumerate(header)))
    return df_full

# Proses hanya data train
df_train = create_dataset_from_files(train_files)

# Tampilkan hasil
df_train.head()

c:\Users\Arief M\Documents\belajar\Semester 5\NLP\Project1\project1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Altering json data train.05.jsonl: 100%|██████████| 14266/14266 [00:00<00:00, 24391.47it/s]


,category,gold_labels,id,paragraphs,source,source_url,summary,news_text,num_of_paragraphs,summary_text,num_of_summary
0,tajuk utama,"{""0"": [false, true], ""1"": [true, true], ""2"": [...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"{""0"": [""Jakarta , CNN Indonesia - - Dokter Rya...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"{""0"": ""Dokter Lula Kamal yang merupakan selebr...","Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",9,Dokter Lula Kamal yang merupakan selebriti sek...,3
1,teknologi,"{""0"": [false, false, false, false], ""1"": [fals...",1509072914-dua-smartphone-zenfone-baru-tawarka...,"{""0"": [""Selfie ialah salah satu tema terpanas ...",dailysocial.id,https://dailysocial.id/post/dua-smartphone-zen...,"{""0"": ""Asus memperkenalkan \u00a0 ZenFone gene...",Selfie ialah salah satu tema terpanas di kalan...,14,Asus memperkenalkan ZenFone generasi keempat...,3
2,hiburan,"{""0"": [true], ""1"": [true], ""2"": [false, false]...",1510613677-songsong-visit-2020-bengkulu-perkua...,"{""0"": [""Jakarta , CNN Indonesia - - Dinas Pari...",cnn indonesia,https://www.cnnindonesia.com/gaya-hidup/201711...,"{""0"": ""Dinas Pariwisata Provinsi Bengkulu kemb...","Jakarta , CNN Indonesia - - Dinas Pariwisata P...",21,Dinas Pariwisata Provinsi Bengkulu kembali men...,2
3,tajuk utama,"{""0"": [true, true], ""1"": [false, false, false]...",1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,"{""0"": [""Merdeka.com - Indonesia Corruption Wat...",merdeka,https://www.merdeka.com/peristiwa/icw-merasa-a...,"{""0"": ""Indonesia Corruption Watch ( ICW ) memi...",Merdeka.com - Indonesia Corruption Watch ( ICW...,5,Indonesia Corruption Watch ( ICW ) meminta Kom...,2
4,tajuk utama,"{""0"": [false, true], ""1"": [true, true, true], ...",1503039338-pembagian-sepeda-usai-upacara-penur...,"{""0"": [""Merdeka.com - Presiden Joko Widodo ( J...",merdeka,https://www.merdeka.com/peristiwa/usai-upacara...,"{""0"": ""Jokowi memimpin upacara penurunan bende...",Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,7,Jokowi memimpin upacara penurunan bendera . Us...,5


In [2]:
! pip install tensorflow tensorflow-gpu


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      Traceback (most recent call last):
        File "C:\Users\Arief M\Documents\belajar\Semester 5\NLP\Project1\project1\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Arief M\Documents\belajar\Semester 5\NLP\Project1\project1\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Arief M\Documents\belajar\Semester 5\NLP\Project1\project1\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _pa


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
from tqdm import tqdm
import tensorflow as tf

# Load Pegasus model dan tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained("thonyyy/pegasus_indonesian_base-finetune")
pegasus_model = PegasusForConditionalGeneration.from_pretrained(
    "thonyyy/pegasus_indonesian_base-finetune",
    from_tf=True  # Memuat model dengan weights TensorFlow
)
# Gunakan GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
pegasus_model = pegasus_model.to(device)

def generate_summary_pegasus(article, max_length, tokenizer, model):
    # Tokenisasi input teks
    input_ids = tokenizer.encode(article, return_tensors='pt', truncation=True)
    input_ids = input_ids.to(device)
    
    # Hasilkan ringkasan
    summary_ids = model.generate(input_ids,
                                 max_length=max_length, 
                                 num_beams=4, 
                                 length_penalty=1.2, 
                                 no_repeat_ngram_size=3, 
                                 early_stopping=True)
    
    # Dekode token menjadi teks
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text

# Tentukan panjang maksimum untuk ringkasan
max_length = df_train['summary_text'].str.len().max()
print("Max summary length:", max_length)

# Batasi jumlah data yang diproses
max_steps = 10  # Sesuaikan dengan kebutuhan
summary_generated = []

# Iterasi untuk memproses data
for i, row in tqdm(df_train[['id', 'news_text']].head(max_steps).iterrows(), total=max_steps):
    sg = generate_summary_pegasus(row['news_text'], max_length, pegasus_tokenizer, pegasus_model)
    summary_generated.append([row['id'], sg])

# Konversi hasil menjadi DataFrame
df_summary_generated = pd.DataFrame(summary_generated)
df_summary_generated = df_summary_generated.rename(columns={0: 'id', 1: 'summary_generated'})

# Gabungkan dengan dataset asli
df_train_result = df_train.head(max_steps).merge(df_summary_generated, on='id')
print(df_train_result.head())

# Evaluasi menggunakan ROUGE
from evaluate import load
rouge = load('rouge')
results = rouge.compute(
    references=df_train_result['summary_text'].values,
    predictions=df_train_result['summary_generated'].values)
print("ROUGE scores:", results)


ModuleNotFoundError: No module named 'tensorflow'